<a href="https://colab.research.google.com/github/shaikadish/imdbProject/blob/main/web_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
%cd drive/MyDrive/GitHub/IMDB_project/imdbProject

/content/drive/MyDrive/GitHub/IMDB_project


In [2]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

     |████████████████████████████████| 968 kB 4.8 MB/s 
     |████████████████████████████████| 138 kB 24.1 MB/s 
     |████████████████████████████████| 359 kB 10.2 MB/s 
     |████████████████████████████████| 55 kB 2.2 MB/s 
     |████████████████████████████████| 4.0 MB 22.9 MB/s 
     |████████████████████████████████| 58 kB 5.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: use options instead of chrome_options
  # Remove the CWD from sys.path while we load stuff.


In [3]:
!pip install beautifulsoup4

In [4]:
import selenium
#webdriver is our tool to interact with the webpage
from selenium import webdriver 
import requests #needed to load the page for BS4
from bs4 import BeautifulSoup
import pandas as pd #Using panda to create our dataframe

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [5]:
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: use options instead of chrome_options
  """Entry point for launching an IPython kernel.


In [6]:
#Set initial empty list for each element:
title = []
link = []
year = []
initial_link_dictionary={'title':[],'link':[],'year':[]}

for review_idx in range(0,16):
  #Set the url link and load the webpage
  # Feature Film/TV Movie, Released between 2010-01-01 and 2022-01-01, English, Including Adult Titles (Sorted by Year Ascending)
  url = f"https://www.imdb.com/search/title/?title_type=feature,tv_movie&release_date=2010-01-01,2022-01-01&languages=en&adult=include&sort=release_date,asc&start={review_idx*5000+1}&ref_=adv_nxt"
  driver.get(url) # THIS DOES NOT WORK, NEED TO GO TO NEXT PAGE A DIFFERENT WAY. URL METHOD DOEESNT WORK AFTER 10,000

  #Grab the block of each individual movie
  block = driver.find_elements_by_class_name('lister-item')
  #Set up for loop to run through all 50 movies on page
  for i in range(0,50):
      
      #Extracting title
      ftitle = block[i].find_element_by_class_name('lister-item-header').text
      #The extracted title has extra elements, so we will have to do some cleaning
      
      #Remove the order in front of the title
      forder = block[i].find_element_by_class_name('lister-item-index').text
      #Extract the year at the end
      fyear = block[i].find_element_by_class_name('lister-item-year').text
      #Drop the order and year and only keep the movie's name
      ftitle=ftitle.replace(" "+fyear,"")
      ftitle = ftitle.replace(forder+' ', '')
      #Then extract the link with cleaned title
      flink = block[i].find_element_by_link_text(ftitle).get_attribute('href')
  #Add item to the respective lists
      initial_link_dictionary['title'].append(ftitle)
      initial_link_dictionary['year'].append(fyear)
      initial_link_dictionary['link'].append(flink)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  
/usr/local/lib/python3.7/dist-packages/selenium/webdriver/remote/webelement.py:446: UserWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  warnings.warn("find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead")
/usr/local/lib/python3.7/dist-packages/selenium/webdriver/remote/webelement.py:265: UserWarning: find_element_by_link_text is deprecated. Please use find_element(by=By.LINK_TEXT, value=link_text) instead
  warnings.warn("find_element_by_link_text is deprecated. Please use find_element(by=By.LINK_TEXT, value=link_text) instead")


In [8]:
initial_link_dictionary
review_idx*5000+1

75001

In [ ]:
#Set an empty list to store user review link
final_link_dictionary={'title':[],'link':[],'year':[],'review_link':[]}
for i,url in enumerate(initial_link_dictionary['link']):
    try:
      #setup user agent for BS4, except some rare case, it would be the same for most browser 
      user_agent = {'User-Agent': 'Requests'}
      #Use request.get to load the whole page
      response = requests.get(url, headers = user_agent)
      #Parse the requests object to BS4 to transform it into html structure
      soup = BeautifulSoup(response.text, 'html.parser')
      #Find the link marked by the USER REVIEWS link text.
      review_link=url.split('/')
      review_link[-1]=soup.find('a', text = 'User reviews').get('href')
      review_link='/'.join(review_link)
      #Append the newly grabed link into its list
      #user_review_links.append(review_link)
      final_link_dictionary['title'].append(initial_link_dictionary['title'][i])
      final_link_dictionary['link'].append(initial_link_dictionary['link'][i])
      final_link_dictionary['year'].append(initial_link_dictionary['year'][i])
      final_link_dictionary['review_link'].append(review_link)
    except:
      continue # For movies with no written reviews

In [20]:
final_link_dictionary['review_link']

['https://www.imdb.com/title/tt1438466/reviews/?ref_=tt_ql_urv',
 'https://www.imdb.com/title/tt1491043/reviews/?ref_=tt_ql_urv',
 'https://www.imdb.com/title/tt1566355/reviews/?ref_=tt_ql_urv',
 'https://www.imdb.com/title/tt1577882/reviews/?ref_=tt_ql_urv',
 'https://www.imdb.com/title/tt1756731/reviews/?ref_=tt_ql_urv',
 'https://www.imdb.com/title/tt1827527/reviews/?ref_=tt_ql_urv',
 'https://www.imdb.com/title/tt1830553/reviews/?ref_=tt_ql_urv',
 'https://www.imdb.com/title/tt1851002/reviews/?ref_=tt_ql_urv',
 'https://www.imdb.com/title/tt1562295/reviews/?ref_=tt_ql_urv',
 'https://www.imdb.com/title/tt1578708/reviews/?ref_=tt_ql_urv',
 'https://www.imdb.com/title/tt1588157/reviews/?ref_=tt_ql_urv',
 'https://www.imdb.com/title/tt1773538/reviews/?ref_=tt_ql_urv',
 'https://www.imdb.com/title/tt1776808/reviews/?ref_=tt_ql_urv',
 'https://www.imdb.com/title/tt1780982/reviews/?ref_=tt_ql_urv',
 'https://www.imdb.com/title/tt1782415/reviews/?ref_=tt_ql_urv',
 'https://www.imdb.com/ti

In [1]:
initial_link_dictionary

NameError: ignored

In [ ]:
link_df=pd.DataFrame(data=final_link_dictionary)
#driver.quit() # tell selenium to close the webpage
link_df

In [1]:
link_df

NameError: ignored

In [117]:
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: use options instead of chrome_options
  """Entry point for launching an IPython kernel.


In [ ]:
#Set list for each element:
title = []
content = []
rating = []
date = []
user_name = []
movie_title=[]
# Step 2, we will grab the data from each user review page
# Use Selenium to go to each user review page. each page is for a different film
for i in range(len(link_df['review_link'])):

  driver.get(link_df['review_link'][i]) # Go to user review page
  driver.implicitly_wait(1) # tell the webdriver to wait for 1 seconds for the page to load to prevent blocked by anti spam software
  
  current_title=link_df['title'][i]

  # LOAD MORE REVIEWS FOR A GIVEN FILM. once loaded, can save reviews
  # Set up action to click on 'load more' button
  # note that each page on imdb has 25 reviews
  page = 1 #Set initial variable for while loop
  #We want at least 1000 review, so get 50 at a safe number
  while page<3:  
      try:
          #find the load more button on the webpage
          load_more = driver.find_element_by_id('load-more-trigger')
          #click on that button
          load_more.click()
          page+=1 #move on to next loadmore button
      except:
          #If couldnt find any button to click, stop
          break
  # After fully expand the page, we will grab data from whole website
  review = driver.find_elements_by_class_name('review-container')

  # save reviews for a given film
  for n in range(0,75):
      try:
          #Some reviewers only give review text or rating without the other, 
          #so we use try/except here to make sure each block of content must has all the element before append them to the list

          #Check if each review has all the elements
          ftitle = review[n].find_element_by_class_name('title').text
          #For the review content, some of them are hidden as spoiler, 
          #so we use the attribute 'textContent' here after extracting the 'content' tag
          fcontent = review[n].find_element_by_class_name('content').get_attribute("textContent").strip()
          frating = review[n].find_element_by_class_name('rating-other-user-rating').text
          fdate = review[n].find_element_by_class_name('review-date').text
          fname = review[n].find_element_by_class_name('display-name-link').text


          #Then add them to the respective list
          title.append(ftitle)
          content.append(fcontent)
          rating.append(frating)
          date.append(fdate)
          user_name.append(fname)
          movie_title.append(current_title)
      except:
          continue

#Build data dictionary for dataframe
data = {'User_name': user_name, 
    'Review title': title, 
    'Review Rating': rating,
    'Review date' : date,
    'Review_body' : content,
    'Movie title': movie_title
    }
#Build dataframe for each movie to export
review = pd.DataFrame(data = data)
#movie = top50['Movie_name'][i] #grab the movie name from the top50 list    
#review['Movie_name'] = movie #create new column with the same movie name column    
#review.to_csv(f'data/{folder_name}/{i+1}.csv') #store them into individual file for each movies, so we can combine or check them later

In [ ]:
review.to_csv(f'review_data.csv')

In [160]:
sum=0
for i in range(len(review['Review Rating'])):
  sum+=int(review['Review Rating'][i].split('/')[0])
average_score=sum/len(review['Review Rating'])
print(average_score)

7.222222222222222


In [132]:
review['Review Rating']

0       5/10
1      10/10
2       6/10
3      10/10
4       9/10
       ...  
447    10/10
448    10/10
449     9/10
450    10/10
451     9/10
Name: Review Rating, Length: 452, dtype: object